In [0]:
dbutils.widgets.text("catalog","")
CATALOG=dbutils.widgets.get("catalog").strip()
dbutils.widgets.text("schema","")
SCHEMA=dbutils.widgets.get("schema").strip()

In [0]:
%python
import json

config_nb_output = dbutils.notebook.run(
    "/Workspace/Users/sangeetha.va@outlook.com/databricks-code-repo/databricks_workouts_2025/4_logistics_usecase/Medallion Arch/generic_medallionArch_Practice/General_conf_utils/configs_path",
    120,{"catalog": CATALOG,"schema": SCHEMA})

config_dict = json.loads(config_nb_output)

CATALOG = config_dict["CATALOG"]
SCHEMA = config_dict["SCHEMA"]
SRC=config_dict["SRC"]
BRONZE = config_dict["BRONZE"]
SILVER = config_dict["SILVER"]
GOLD = config_dict["GOLD"]
SILVERDB = config_dict["SILVERDB"]
GOLDDB = config_dict["GOLDDB"]

In [0]:
%run "/Workspace/Users/sangeetha.va@outlook.com/databricks-code-repo/databricks_workouts_2025/4_logistics_usecase/Medallion Arch/generic_medallionArch_Practice/General_conf_utils/util_functions"

In [0]:
from pyspark.sql.window import Window

df = spark.read.format("delta").load(f"{SILVER}/shipments")
print(df.columns)

# TOP 3 DRIVERS BY COST PER HUB
w = Window.partitionBy("source_city") \
          .orderBy(F.col("shipment_cost").desc())

top3 = df.withColumn("rank", F.dense_rank().over(w)) \
         .filter("rank <= 3")

# LEAD / LAG
lag_df = df.withColumn(
    "prev_shipment_days",
    F.lag("shipment_year").over(
        Window.partitionBy("shipment_id").orderBy("shipment_year")
    )
)

In [0]:
write_file(top3,f"{GOLD}/top3_drivers")
write_file(lag_df,f"{GOLD}/prev_shipment_days")